<a id="title"></a>
# Convolutional Neural Network MNIST Classification Tutorial using PyTorch
***
## Learning Goals:
By the end of this tutorial, you will:
- learn the basics of using the PyTorch library
- build a convolutional neural network (CNN) to classify MNIST images
- train and evaluate a CNN
- analyze performance on false positive/negative instances

## Table of Contents
[Introduction](#intro) <br>
[0. Imports](#imports) <br>
[1. MNIST Dataset and Scaling](#mnist) <br>
[2. Build a CNN Classifier](#build) <br>
[3. Test Model Functionality](#test) <br>
[4. Set Training and Test Sets](#set) <br>
[5. Hyperparameters and Loading](#hyper) <br>
[6. Train Model](#train) <br>
[7. Plot Loss Function and Accuracy](#plot) <br>
[8. Analyze Samples](#analyze) <br>
[9. Conclusions](#con) <br>
[Additional Resources](#add) <br>
[About this Notebook](#about) <br>
[Citations](#cite) <br>

## Introduction <a id="intro"></a>

The main purpose of this notebook is to teach basic functionality of [PyTorch](https://pytorch.org/), a deep learning Python library. This tutorial is not an exhaustive introduction to machine learning and assumes the user is familiar with vocabulary (supervised v unsupervised, neural networks, loss functions, backpropogation, etc) and methodology (model selection, feature selection, hyperparameter tuning, etc). Look at [Additional Resources](#add) for more complete machine learning guides. The paragraphs below serve as a brief refresher.

Machine learning (ML) is a set of data driven algorithms in which a model can be trained to make predictions based off of past experiences. There are two main regimes of ML: supervised learning and unsupervised learning. Supervised learning techniques, such as regression and classification, require labels in order to be trained. Unsupervised learning techniques, such as clustering and dimensionality reduction, do not require labels to be trained.

Neural networks are a subset of models that learn to extract features useful for training. Convolutional neural networks (CNNs) are neural networks that are excellent in computer vision, i.e. solving problems involving images. A CNN convolves patches of an image to learn filters that are useful for training. These networks became more accessible by [AlexNet](https://en.wikipedia.org/wiki/AlexNet) in 2012.

[MNIST](http://yann.lecun.com/exdb/mnist/) is a dataset of 70,000 (60,000 train; 10,000 test) of handwritten digits from 0-9 (inclusive). Each image is 28 pixels by 28 pixels, each pixel being an 8-bit integer (0-255 integer gray scaled). The dataset is small compared to real world datasets used in industry, but is clean and large enough to use for efficient modeling. In addition, because of how well defined the features and labels are, it has become a staple in learning and testing different algorithms dealing with images.

**In this notebook, we will build a CNN using PyTorch to classify MNIST images of handwritten digits.**

## 0. Imports <a id="imports"></a>

If you are running this notebook on Google Colab, you shouldn't have to install anything. If you are running this notebook in Jupyter, this notebook assumes you created the virtual environment defined in `environment.yml`. If not, close this notebook and run the following lines in a terminal window:

`conda env create -f environment.yml`

`conda activate pytorch_mnist_cnn`

We import the following libraries:
- *numpy* for handling arrays
- *matplotlib* for plotting
- *tqdm* for keeping track of loop speed
- *tensorflow* for accessing MNIST images 
- *torch* as our machine learning framework

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

import tensorflow as tf

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

## 1. MNIST Dataset and Scaling<a id="mnist"></a>

The MNIST dataset is nicely packed in `tensorflow` as `np.arrays`, which is why we are grabbing our data from there instead of directly from `torch`. The data is unpacked as `x_train` for training features, `y_train` for training labels, `x_test` for testing features, and `y_test` for testing labels.

In [ ]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

We will check to make sure the data is consistent with what was described in the [Introduction](#intro).

In [ ]:
print ('The shape of the training features is {} corresponding to '
       '(# of samples, pixel length, pixel width). \n'.format(x_train.shape))
print ('The shape of the training labels is {} corresponding to '
       '(# of samples). \n'.format(y_train.shape))
print ('The shape of the testing features is {} corresponding to '
       '(# of samples, pixel length, pixel width). \n'.format(x_test.shape))
print ('The shape of the testing labels is {} corresponding to '
       '(# of samples). \n'.format(y_test.shape))
print ('The min and max pixel values are {} and {}, respectively, as {}. \n'.format(x_train.min(), 
                                                                                    x_train.max(), 
                                                                                    x_train.dtype))
print ('The labels are type {}. \n'.format(y_train.dtype))

Now we can take a look at the contents of the dataset. Below is the first sample of the training set, the number 5. Feel free to change the index to view other images.

In [ ]:
index = 0
plt.title('Training Image {} with a label of {}'.format(index, y_train[index]))
plt.imshow(x_train[index])
plt.colorbar()
plt.show()

We'll also define some frequently used global variables. `x_train_size` is the number of images in the training set, `x_test_size` is the number of images in the test set, and `x_length` is the length/width of an image.

In [ ]:
x_train_size = x_train.shape[0]
x_test_size = x_test.shape[0]
x_length = x_train.shape[1]

print (x_train_size, x_test_size, x_length)

In machine learning, it is common practice to normalize data into more reasonable values in order to optimize calculating gradients in our loss function and train more robustly. We min-max scale our images to have a minimum value of 0 and a maximum value of 1. The normalization factor is defined as the maximum pixel value (255) and used to define our scaled variables.

In [ ]:
norm = x_train.max()

x_train_scale = x_train / norm
x_test_scale = x_test / norm

We can look at our previous example to confirm the image was properly scaled.

In [ ]:
index = 0
plt.title('Training Scaled Image {} with a label of {}'.format(index, y_train[index]))
plt.imshow(x_train_scale[index])
plt.colorbar()
plt.show()

## 2. Build a CNN Classifier <a id="build"></a>

PyTorch has its own unique data objects called `torch.utils.data.Dataset`. `Dataset` has functions to retrieve the data length and instances. The datasets built from the class are used as inputs for `torch.utils.data.Dataloader`, which prepares our data for training.

In [ ]:
class LoadDataset(Dataset):
    
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        
        return image, label

Here we define the functions and layers to build our two-layer CNN. The constructor has our model hyperparameters as inputs:

- `filters`: the number of filters the CNN will learn
- `neurons`: the numbers of output neurons in the fully connected layers
- `sub_array_size`: the image length/width
- `k`: the length/width of the filter being learned
- `pool`: the length/width of max pooling
- `pad`: the length/width of zero padding

Using the constructor's parameters, we define the CNNs layers and functions. We use the [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) as our activation function to add nonlinearity to our model, use max pool to downsample (decrease model size and forces better feature extraction), use `filters`, `k`, and `pad` to build our CNN layers, and use `neurons` to build our fully connected layers.

The `forward` fucntion organizes our CNN from the functions we defined in the constructor. The CNN is built as follows:
- Convolutional Layer 1
    - convole 1 28x28 image into 16 28x28 feature maps
    - activate the feature maps using ReLU
    - max pool the 16 28x28 feature maps to 16 14x14 feature maps
- Convolutional Layer 2
    - convole 16 14x14 feature maps into 32 14x14 feature maps
    - activate the feature maps using ReLU
    - max pool the 32 14x14 feature maps to 32 7x7 feature maps
- Flatten the 32 7x7 feature maps to a 1D 32 * 7 * 7 array
- Fully Connected Layer 1
    - use the flatten 1D array as inputs for the 64 neuron hidden layer
    - activate the neurons using ReLU
- Fully Connected Layer 2
    - use the 64 neuron hidden layer as inputs for the 10 output neurons
    
For deeper networks, we can regularize our network using [dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) and [batch normalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html). We choose to omit these techniques to have a simpler model.
    
**Note: the output is activated automatically in our loss function (see [Section 5](#hyper)).**

In [ ]:
# flatten torch tensor
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# define functions and build model
class Classifier(nn.Module):
    def __init__(self, 
                 filters = [1, 16, 32],        # most people use powers of 2
                 neurons = [64, 10],           # neurons of fully connected layer
                 sub_array_size = x_length,    # image size (28x28)
                 k = 3,                        # kernel size (3x3)
                 pool = 2,                     # pooling size (2x2)
                 pad = 1):                     # padding length (1 pixel border)

        super(Classifier, self).__init__()

        # The Rectified Linear Unit (ReLU)
        self.relu = nn.ReLU()

        # Max Pool
        self.mp = nn.MaxPool2d(pool, return_indices=False)
        
        # Flattens the feature map to a 1D array
        self.flatten = Flatten()

        # ---- CONVOLUTION  ----
        self.conv1 = nn.Conv2d(in_channels=filters[0], out_channels=filters[1], kernel_size=k, padding=pad)
        self.conv2 = nn.Conv2d(in_channels=filters[1], out_channels=filters[2], kernel_size=k, padding=pad)
        
        # Total number of pixels in the input of first connected layer:
        # number of pooling layers = number of convolutional layers - 1
        # total amount of pooling = pooling size ** number of pooling layers
        # length/width of feature map after last pooling = sub array size / 
        #                                                  total amount of pooling
        # total number of feature map pixels = feature map length/width ** 2
        # total number of input neurons = number of filters in final layer * 
        #                                 total number of feature map pixels
        
        num_pool = len(filters) - 1
        neurons_flat = filters[-1] * (sub_array_size // pool ** num_pool) ** 2 
        
        # ---- FULLY CONNECTED ----
        self.fc1 = nn.Linear(neurons_flat, neurons[0])
        self.fc2 = nn.Linear(neurons[0], neurons[1])

    def forward(self,x):
        
        # Convolutional Layer 1
        x = self.conv1(x)
        x = self.relu(x)
        x = self.mp(x)

        # Convolutional Layer 2
        x = self.conv2(x)
        x = self.relu(x)
        x = self.mp(x)

        # Flatten Layer
        x = self.flatten(x)

        # Fully Connected 1
        x = self.fc1(x)
        x = self.relu(x)

        # Fully Connected 2
        x = self.fc2(x)      

        return x

## 3. Test Model Functionality <a id="test"></a>

Before training, we need to make sure our model is properly built, i.e. the expected input (2D 28x28 array) will return the expected output (2D 1x10 array). An error indicates the architecture is inconsistent in some way, such as unexpected input and output filters, unexpected input and output neurons, etc. Some ways to "break" the model are listed below:
- comment out a method in the constructor
- manually change `neurons_flat` to a different value
- manually change `out_channels` in `self.conv1` or `self.conv2` to a different value

To start off, we will build our model.

In [ ]:
model = Classifier()

Next, we change the shape of our image to be compatible with PyTorch. The input dimensions for images are (number of samples, number of input channels, y dimension, x dimension), which in our case is (1, 1, 28, 28).

In [ ]:
index = 0
test_image = x_train_scale[index].reshape(1,1,x_length,x_length) 

After the dimensions are changed, we convert the image from a `np.array` to a `torch.Tensor`.

In [ ]:
test_image_torch = torch.Tensor(test_image)

Now we can "predict" the output neurons of the input image.

In [ ]:
testoutput_torch = model(test_image_torch)

Since there is no error, we know our model is working. We also move the output from our model using the `detach()` method and convert the `torch.Tensor` to a `np.array` by using the `numpy()` method.

In [ ]:
testoutput = testoutput_torch.detach().numpy()

Let's check the shape of the output neurons to make sure they are what we expect.

In [ ]:
print ('The shape of the output neurons are {}.'.format(testoutput.shape))

Let's return the output neurons. Note since we do not use an activation function at the end of our network, the domain of our output can be any real number.

In [ ]:
testoutput

We can use the [softmax activation function](https://en.wikipedia.org/wiki/Softmax_function) to convert the output neurons to probabilities for each classification with the index corresponding to the digit classification probability, e.g. the 0th index corresponds to the probability of a 0 classification. Since our model isn't trained, all the output probabilities are approximately 0.1, indicating our model is randomly "classifying".

In [ ]:
softmax = nn.Softmax(dim=1)
softmax(testoutput_torch)

In addition, it's good practice to know how many trainable parameters are in our model. The number of trainable parameters can be used as a proxy for estimating total training time. We define [a counting function](https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model) for us and determine how many trainable parameters there are in our model.

In [ ]:
def count_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        print([name, param])
        total_params+=param
    return total_params

In [ ]:
count_parameters(model)

## 4. Set Training and Test Sets <a id="set"></a>

PyTorch uses iterables to create its data objects. Here we show two ways to format the data to be PyTorch compatible.

1. **Use lists:** Experienced Python users are more likely to be comfortable using and manipulating lists. The function `format_dataset` makes a 2D list with elements being [image, label]. The image is reshaped to (1, 28, 28) so the first dimension corresponds to number of input channels.

2. **Use LoadDataset:** In [Section 2](#build), we defined the `LoadDataset` class to format the data to be PyTorch compatible. The `Dataset` class comes with additional functionality specifically for PyTorch, but is beyond the scope of this tutorial.

We choose option 1 as default, but option 2 can be uncommented below. Using either does not affect training at all and is up to user comfortability/preference.

In [ ]:
def format_dataset(image_set, labels, size=x_length):
    data_set = []
    for i in range(len(image_set)):
        data_set.append([image_set[i].reshape(1,size,size), labels[i]])  
    return data_set

In [ ]:
train_set = format_dataset(x_train_scale, y_train)
val_set = format_dataset(x_test_scale, y_test)

In [ ]:
# LoadDataset class
#train_set = LoadDataset(x_train_scale.reshape(x_train_size,1,x_length,x_length), y_train)
#val_set = LoadDataset(x_test_scale.reshape(x_test_size,1,x_length,x_length), y_test)

## 5. Hyperparameters and Loading <a id="hyper"></a>

First, we must set our hyperparameters for the model to use for training. The hyperparamters we are using are batch size, shuffle, and number of workers. Batch size can be tuned as needed to improve results. Shuffle should almost always be True since the data shouldn't be ordered in any specific way when training. In addition, the number of workers has a default of 0, which uses the main processor on the machine you are using.

In [ ]:
params = {
        'batch_size': 128,
        'shuffle': True,
        'num_workers': 0
    }

Next, we choose the number of epochs we wish to train for.

In [ ]:
num_epochs = 5

Another useful metric to know is how many updates our model will perform during training. We can calculate this by finding the number of batches in the training set (number of training samples / batch size) and multiplying it by the number of epochs. Knowing how many batches our model might need to be well trained can be a good place to start when tuning hyperparameters.

In [ ]:
print ('The model will train using a total of {} batches'.format(num_epochs * 
                                                       int(x_train_scale.shape[0] / params['batch_size'])))

With our hyperparameters set, we can load our training and test set using `DataLoader`. 

**Note the variable and function names in the notebook are directed for validation sets, but we will use them for the test set instead.** That being said, we use the definitions for validation set and test set interchangeably here.

In [ ]:
# TRAINING SET
train_loader = DataLoader(train_set, **params)

# TEST SET
valid_loader = DataLoader(val_set, **params)

We will initialize our model again to be sure we are starting from scratch.

In [ ]:
model = Classifier()

Now we define our loss function to be [Cross Entropy Loss](https://en.wikipedia.org/wiki/Cross_entropy), which combines [softmax and the negative log likelihood loss](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/). This function is standard in multiclass classification problems.

In [ ]:
distance = nn.CrossEntropyLoss()

Then we choose our optimizer to be [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam), since the learning rate updates automatically and trains relatvely fast compared to [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent).

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),  weight_decay=1e-5)

If you have GPUs available, then those will be used for training. If not, then the model will train on CPUs.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device);

Let's print the device to make sure we know what's available.

In [ ]:
device

## 6. Train Model <a id="train"></a>

In order to train our model, we have to manually loop through our data for training. This is probably the biggest difference between PyTorch and [Tensorflow](https://www.tensorflow.org/), but this allows for more hands-on manipulation of how training is performed, which can be advantageous. We will train our model as follows:
1. Change the model to trianing mode to activate backpropogation
2. Initialize training loss to be 0
3. Loop through each batch of features and labels by:
    - Putting the data onto your device
    - Calculating the output neurons and the loss
    - Performing backgrpopogation and adding the batch training loss to total training loss
4. Normalize the total training loss by number of samples

In [ ]:
# Define train loop

def train_model(train_loader):

    # Change model to training mode (activates backpropogation)
    model.train()
    
    # Initialize training loss
    train_loss = 0
    
    # Loop through batches of training data
    for data, target in train_loader:
        
        # Put training batch on device
        data = data.float().to(device)
        target = target.type(torch.LongTensor).to(device)

        # Calculate output and loss from training batch
        output = model(data)
        loss = distance(output, target)
        
        # Backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Normalize training loss from one epoch
    train_loss_norm = train_loss / len(train_loader)
    
    return train_loss_norm

In addition, we define a similar loop for evaluating the test set at each epoch, which signals us if our model is generalizing. We will test our model as follows:
1. Change model to evaluation mode to deactivate backpropogation
2. Initialize test loss and number of correctly classified samples to 0
3. Loop through each batch of features and labels by:
    - Putting the data onto your device
    - Calculating the output neurons and the loss
    - Counting the number of correct predictions
4. Calculate test set accuracy
5. Normalize the total test loss by number of samples

In [ ]:
# Define validation loop

def validate_model(valid_loader):

    # Change model to evaluate mode (deactivates backpropogation)
    model.eval()
    
    # Initialize validation loss and number of correct predictions
    val_loss = 0
    correct = 0
    
    # Do not calculate gradients for the loop
    with torch.no_grad():
        
        # Loop through batches of validation data
        for data, target in valid_loader:
            
            # Put validation batch on device
            data = data.float().to(device)
            target = target.type(torch.LongTensor).to(device)
            
            # Calculate output and loss from validation batch
            output = model(data)
            val_loss += distance(output, target).item()
            
            # Count number of correct predictions
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        
        # Calculate accuracy
        accuracy = 100. * correct / len(valid_loader.dataset)
    
    # Normalize validation loss from one epoch
    val_loss_norm = val_loss / len(valid_loader)
    
    return val_loss_norm, accuracy

Finally, we can train our model! We will print out the train and test loss/accuracy per epoch to keep track of performance. The loop below performs the training and validation loops defined above and records our metrics.

In [ ]:
# keep track of metrics
lst_train_loss = []
lst_val_loss = []
lst_accuracy = []

# training loop
for epoch in tqdm(range(num_epochs), total=num_epochs):

    # Go through loops
    train_loss = train_model(train_loader)
    val_loss, accuracy = validate_model(valid_loader)

    # Append metrics
    lst_train_loss.append(train_loss)
    lst_val_loss.append(val_loss)
    lst_accuracy.append(accuracy)

    # Log
    print('Epoch {:.3f} - Train loss: {:.3f} - Val Loss: {:.3f} - Accuracy: ({:.0f}%)'.format(
            epoch, train_loss, val_loss, accuracy))

## 7. Plot Loss Function and Accuracy <a id="plot"></a>

We plot the train/test loss and test accuracy to determine how well converged our model is.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=[10,5])

axs[0].set_title('Loss')
axs[0].plot(np.arange(num_epochs), lst_train_loss, label='train')
axs[0].plot(np.arange(num_epochs), lst_val_loss, label='val')
axs[0].set_xlabel('Epochs')
axs[0].legend()

axs[1].set_title('Accuracy')
axs[1].plot(np.arange(num_epochs), lst_accuracy, color='C1')
axs[1].set_xlabel('Epochs')

## 8. Analyze Samples <a id="analyze">
    
Now that our model is trained, let's analyze some samples to see the classification probabilities of some images.
    
First, we predict the output neurons and labels of our test set.

In [ ]:
output = model(torch.Tensor(x_test_scale.reshape(x_test_size, 1, x_length, x_length)))

val_pred = output.data.max(1, keepdim=True)[1].detach().numpy().flatten()

Next, we can look at random examples in our test set and plot classification probabilities using a bar graph.

In [ ]:
# choose random image and corresponding output neurons from test set
rand_index = np.random.randint(x_test_scale.shape[0])
rand_image = x_test_scale[rand_index]
rand_class_prob_tensor = softmax(val_pred[rand_index:rand_index+1])
rand_class_prob = rand_class_prob_tensor.detach().numpy().flatten()

# plot image and classification probabilities
fig, axs = plt.subplots(1,2,figsize=[10,5])
axs[0].set_title('Testing Scaled Image {} with a label of {}'.format(rand_index, y_test[rand_index]))
axs[0].imshow(rand_image)
axs[1].set_title('Classification Probabilities for Testing Scaled Image {}'.format(rand_index))
axs[1].bar(np.arange(10), rand_class_prob)
axs[1].set_xlabel('Label')
axs[1].set_ylabel('Probability (Log Scale)')
axs[1].set_yscale('log')
print ('Prediction: {}, Label: {}'.format(val_pred[rand_index], y_test[rand_index]))
plt.tight_layout()

In addition we can look at false positive/negative samples to investigate harder samples in the test set.

We create a mask that will return incorrect predictions.

In [ ]:
mask = val_pred != y_test

In [ ]:
# choose random incorrect image and corresponding output neurons from test set
false_index = np.random.randint(x_test_scale[mask].shape[0])
false_image = x_test_scale[mask][false_index]
false_class_prob = (nn.Softmax(dim=1)(output[mask][false_index:false_index+1])).detach().numpy().flatten()

# plot image and classification probabilities
fig, axs = plt.subplots(1,2,figsize=[10,5])
axs[0].set_title('Testing Scaled Image {} with a label of {}'.format(false_index, y_test[mask][false_index]))
axs[0].imshow(false_image)
axs[1].set_title('Classification Probabilities (Log Scale)')
axs[1].bar(np.arange(10), false_class_prob)
axs[1].set_yscale('log')
print ('Prediction: {}, Label: {}'.format(val_pred[mask][false_index], y_test[mask][false_index]))

## 9. Conclusions <a id="con"></a>

Thank you for walking through this notebook. Now you should be more familiar with:
- the basics of using the PyTorch library
- building a convolutional neural network (CNN)
- training and evaluating a CNN
- analyzing performance on samples, such as false positive/negative instances

**Congratulations, you have completed the notebook!**

## Additional Resources <a id="add"></a>

Machine learning is a dense and rapidly evolving field of study. Becoming an expert takes years of practice and patience, but hopefully this notebook brought you closer in that direction. Here are some of the author's favorite resources for learning about machine learning and data science:

- [Google Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/ml-intro)
    - [CNN Practica](https://developers.google.com/machine-learning/practica/image-classification)
- [scikit-learn Python Library](https://scikit-learn.org/stable/index.html) (go-to for most ML algorithms besides neural networks)
- [StatQuest Youtube Channel](https://www.youtube.com/c/joshstarmer)
- [DeepLearningAI YouTube Channel](https://www.youtube.com/c/Deeplearningai/videos)
- [Towards Data Science](https://towardsdatascience.com/) (articles about data science and machine learning, some invlovling example blocks of code)
- Advance searching [arxiv](https://arxiv.org/search/advanced) (e.g. search term "machine learning" in Abstract for Subject astro-ph) to see what others are doing currently
- Google, YouTube, and Wikipedia in general

## About this Notebook <a id="about"></a>

**Author:** Fred Dauphin, DeepWFC3

**Updated on:** 2021-12-03

## Citations <a id="cite"></a>

If you use `numpy`, `matplotlib`, or `torch` for published research, please cite the authors. Follow these links for more information about citing `numpy`, `matplotlib`, and `torch`:

* [Citing `numpy`](https://numpy.org/doc/stable/license.html)
* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/license.html#:~:text=Matplotlib%20only%20uses%20BSD%20compatible,are%20acceptable%20in%20matplotlib%20toolkits.)
* [Citing `torch`](https://github.com/pytorch/pytorch/blob/master/LICENSE)

***
[Top of Page](#title)